# `nx_merge_parallel_edges()` with [`cityseer`](https://github.com/benchmark-urbanism/cityseer-api)

* **Simons, G.** (2023). *The cityseer Python package for pedestrian-scale network-based urban analysis*. Environment and Planning B: Urban Analytics and City Science, 50(5), 1328-1344. https://doi.org/10.1177/23998083221133827

-----------------

1. Setup
2. Prep `cityseer` graph
3. Experiment
4. Viz

-----------------

## 1. Setup

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-06-12T17:40:23.976317-04:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.25.0

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import cityseer  # noqa: F401
import geopandas
import momepy
import networkx
import pyproj
import core
from cityseer.tools import graphs, io

%watermark -w
%watermark -iv

Watermark: 2.4.3

core     : 0.1.dev99+gb02958b.d20240608
momepy   : 0.7.1.dev43+g6f1d740
cityseer : 4.12.0
pyproj   : 3.6.1
networkx : 3.3
geopandas: 0.14.4



In [3]:
graphs.nx_merge_parallel_edges?

Signature:
graphs.nx_merge_parallel_edges(
    nx_multigraph: 'MultiGraph',
    merge_edges_by_midline: 'bool',
    contains_buffer_dist: 'int',
) -> 'MultiGraph'
Docstring:
Check a MultiGraph for duplicate edges; which, if found, will be merged.

The shortest of these parallel edges is selected and buffered by `contains_buffer_dist`. If this buffer contains an
adjacent edge, then the adjacent edge is merged. Edges falling outside this buffer are retained.

When candidate edges are found for merging, they are replaced by a single new edge. The new geometry selected from
either:
- An imaginary centreline of the combined edges if `merge_edges_by_midline` is set to `True`;
- Else, the shortest edge is retained, with longer edges discarded.

Parameters
----------
nx_multigraph: MultiGraph
    A `networkX` `MultiGraph` in a projected coordinate system, containing `x` and `y` node attributes, and `geom`
    edge attributes containing `LineString` geoms.
merge_edges_by_midline: bool
    Wheth

### Funtions

In [4]:
def merge_parallel_routine(
    graph: networkx.Graph, midline: bool, buff: int | float
) -> networkx.Graph:
    # "simple geoms"
    _g = graphs.nx_simple_geoms(graph.copy())
    print(core.utils.graph_size(f"simple geoms: {city}", _g))

    # "remove filler nodes"
    _g = graphs.nx_remove_filler_nodes(_g.copy())
    print(core.utils.graph_size(f"remove filler nodes: {city}", _g))

    # run ``nx_merge_parallel_edges``
    _g = graphs.nx_merge_parallel_edges(_g.copy(), midline, buff)
    print(core.utils.graph_size(f"merged parallel edges\n  {buff=}m", _g))

    return _g

In [5]:
def convert_cs2mm(
    graph: networkx.Graph, crs: pyproj.CRS
) -> tuple[geopandas.GeoDataFrame, geopandas.GeoDataFrame]:
    """conversion from ``cityseer`` format to ``momepy``/``geopandas``"""

    cs_gdf = io.geopandas_from_nx(graph, crs=crs).rename_geometry("geometry")
    mm_nx = momepy.gdf_to_nx(cs_gdf, integer_labels=True)

    n, e = momepy.nx_to_gdf(mm_nx)
    n = n[["nodeID", "geometry"]].copy()
    e = e[["node_start", "node_end", "geometry"]].copy()

    return n, e

-----------------------------------------
## 2. Prep `cityseer` graph

* https://cityseer.benchmarkurbanism.com/tools/graphs#nx-merge-parallel-edges

In [6]:
# city = "Douala"
city = "Liège"
core.utils.city_fua[city]

1656

In [7]:
roads_bare = core.utils.read_parquet_roads(core.utils.city_fua[city])
orig_crs = roads_bare.crs

### Convert edges `geopandas.GeoDataFrame` to `cityseer`-compatible `networkx.Graph`
* performs an initial 1-meter parallel edge buffer

In [8]:
G = io.nx_from_generic_geopandas(roads_bare)
print(core.utils.graph_size(f"NetworkX Graph from CitySeer: {city}", G))

INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|███████████████████████████████| 105174/105174 [00:00<00:00, 270345.20it/s]

NetworkX Graph from CitySeer: Liège
	* MultiGraph with 98777 nodes and 105156 edges


--------------------------------------

## 3. Experiement

### params
* Merge parallel edges by 2 methods: a derived centerline and the shortest edge
* Use a buffer of 5m to 100m, by 5m

In [9]:
merge_edges_by_midline = [True, False]
contains_buffer_dists = list(range(5, 101, 5))

In [10]:
results = {}

for merge_midline in merge_edges_by_midline:
    results[merge_midline] = {}
    for contains_buffer_dist in contains_buffer_dists:
        results[merge_midline][contains_buffer_dist] = {}

        # simple geoms >> remove filler nodes >> merge parallel edges
        _G = merge_parallel_routine(G.copy(), merge_midline, contains_buffer_dist)

        # conversion from ``cityseer`` format to ``momepy``/``geopandas``
        nodes, edges = convert_cs2mm(_G, orig_crs)

        results[merge_midline][contains_buffer_dist]["nodes"] = nodes
        results[merge_midline][contains_buffer_dist]["edges"] = edges

INFO:cityseer.tools.graphs:Generating interpolated edge geometries.
100%|███████████████████████████████| 105156/105156 [00:00<00:00, 203681.33it/s]


simple geoms: Liège
	* MultiGraph with 98777 nodes and 105156 edges


INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████████████████████████████| 98777/98777 [00:04<00:00, 23148.14it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 5.


remove filler nodes: Liège
	* MultiGraph with 22216 nodes and 28595 edges


100%|█████████████████████████████████| 28595/28595 [00:00<00:00, 416727.03it/s]
INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.


...

simple geoms: Liège
	* MultiGraph with 98777 nodes and 105156 edges


INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████████████████████████████| 98777/98777 [00:04<00:00, 23117.46it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 100.


remove filler nodes: Liège
	* MultiGraph with 22216 nodes and 28595 edges


100%|█████████████████████████████████| 28595/28595 [00:00<00:00, 398182.40it/s]
INFO:cityseer.tools.io:Preparing node and edge arrays from networkX graph.


merged parallel edges
  buff=100m
	* MultiGraph with 22216 nodes and 28150 edges


---------------------------------

## 4. Viz

### use cases

In [11]:
points, fpath_base = core.utils.load_usecases(city)

In [12]:
points

{'roundabout': {'coords': [5.587873, 50.641955],
  'address': 'Place du Congrès',
  'comments': '...'},
 'highway': {'coords': [5.560604, 50.632068],
  'address': 'A602 / Rue de Puisatiers',
  'comments': '...'},
 'parkinglot': {'coords': [5.592738, 50.616253],
  'address': 'Carrefour Quai des Vennes',
  'comments': '...'},
 'intersection': {'coords': [5.511301, 50.618373],
  'address': 'Pont de Seraing',
  'comments': '...'},
 'parallel_edges_1': {'coords': [5.5708487, 50.6236314],
  'address': 'LIEGE Place des Franchises',
  'comments': 'small, textbook case'},
 'parallel_edges_2': {'coords': [5.6192988, 50.6928202],
  'address': "WBCC, Rue d'Abhooz 82, 4040 Herstal, Belgium",
  'comments': 'parking lot of parallel edges'},
 'parallel_edges_3': {'coords': [5.6202498, 50.6790271],
  'address': 'Rue du Coq Mosan 7, 4040 Herstal, Belgium',
  'comments': 'collapsed neighborhood - parallel edges'}}

In [13]:
fpath_base

PosixPath('/Users/jgaboardi/github_repos/uscuni/simplification/usecases/1656')

In [14]:
package = "cityseer"
fpath_pack = fpath_base / package
fpath_pack

PosixPath('/Users/jgaboardi/github_repos/uscuni/simplification/usecases/1656/cityseer')

### class experiments

In [15]:
contexts = ["parallel_edges_1", "parallel_edges_2", "parallel_edges_3"]
contexts

['parallel_edges_1', 'parallel_edges_2', 'parallel_edges_3']

In [16]:
for context in contexts:
    # isolate central AOI
    point = points[context]["coords"]
    center = core.viz.context.location(point, orig_crs)

    for merge_midline in merge_edges_by_midline:
        # make subfolder for plot saving
        _context = f"{context}_midline_{merge_midline}"
        fpath_context = core.viz.context.path(_context, fpath_pack)

        for contains_buffer_dist in contains_buffer_dists:
            # Make class-param plot
            buff_pad_3 = f"{contains_buffer_dist:03d}"
            fmt_title = (
                f"Contains Buffer Dist: {buff_pad_3}m\n"
                f"Merge Edges by Midline: {merge_midline}"
            )
            fmt_fname = f"{buff_pad_3}_{merge_midline}"
            core.viz.context.param_plot(
                results[merge_midline][contains_buffer_dist]["nodes"],
                results[merge_midline][contains_buffer_dist]["edges"],
                center,
                fmt_title,
                fmt_fname,
                fpath_context,
                orig_crs,
            )

        # Make Video
        core.viz.context.video(fpath_context)

-----------------------------------